In [1]:
from pyspark.sql import SparkSession

In [2]:
spark =SparkSession.builder.appName("Mock Session").getOrCreate()
print(spark)

25/03/23 17:32:12 WARN Utils: Your hostname, TTNPL-8203 resolves to a loopback address: 127.0.1.1; using 192.168.163.68 instead (on interface wlp0s20f3)
25/03/23 17:32:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/23 17:32:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
rdd = spark.sparkContext.parallelize([("Brooke", 20), ("Denny", 31), ("Jules", 30), ("TD", 35), ("Brooke", 25)])

In [7]:
rdd2  =rdd.reduceByKey(lambda x,y:max(x,y))
rdd2.collect()

[('Brooke', 25), ('Denny', 31), ('TD', 35), ('Jules', 30)]

In [8]:
rdd3 = rdd.aggregateByKey(
    (0,0),
    lambda acc , value: (acc[0]+value , acc[1]+1),
    lambda acc1,acc2 : (acc1[0]+acc2[0],acc2[1]+acc1[1])
)

In [13]:
rdd4 = rdd3.mapValues(lambda x:x[0]/x[1])

In [14]:
dataFrame =  rdd4.toDF(["name","avg_age"])

In [15]:
dataFrame.show()

+------+-------+
|  name|avg_age|
+------+-------+
|Brooke|   22.5|
| Denny|   31.0|
|    TD|   35.0|
| Jules|   30.0|
+------+-------+



In [17]:
df = spark.read.csv(path = "file:///home/hdoop/notebooks/data/test_assessment_mock",inferSchema = True,header=True)

In [18]:
df.createTempView("temp")

In [20]:
df.printSchema()

root
 |-- date: integer (nullable = true)
 |-- delay: integer (nullable = true)
 |-- distance: integer (nullable = true)
 |-- origin: string (nullable = true)
 |-- destination: string (nullable = true)



In [28]:
spark.sql('''select min(delay) as min_delay,
        max(delay) as max_delay,
        origin
        from temp
        group by origin''' ).show(truncate = False,n=10)

[Stage 35:===========================================>              (6 + 2) / 8]

+---------+---------+------+
|min_delay|max_delay|origin|
+---------+---------+------+
|-15      |401      |GEG   |
|-27      |428      |BUR   |
|-19      |414      |GRB   |
|-23      |435      |GTF   |
|-21      |403      |GRR   |
|-21      |353      |EUG   |
|-27      |745      |GSO   |
|-14      |135      |BTM   |
|-15      |196      |COD   |
|-30      |530      |FAR   |
+---------+---------+------+
only showing top 10 rows



In [32]:
c = df.select(df.origin).distinct().count()

In [33]:
print(c)

255


In [42]:
from pyspark.sql.functions import max,min,col,expr

In [40]:
max_min_delay_df = df.groupby(df.origin).agg(
    
    max("delay").alias("max_delay"),
    min("delay").alias("min_delay")
)

In [47]:
max_min_delay_df2 = max_min_delay_df.withColumn("adjusted_max_delay",1.5*col("max_delay"))

In [48]:
from pyspark.sql.functions import md5

In [49]:
df2  = max_min_delay_df2.withColumn("enc_origin",md5("origin"))

In [50]:
df2.printSchema()

root
 |-- origin: string (nullable = true)
 |-- max_delay: integer (nullable = true)
 |-- min_delay: integer (nullable = true)
 |-- adjusted_max_delay: double (nullable = true)
 |-- enc_origin: string (nullable = true)



In [51]:
df2.drop(df.origin)

DataFrame[max_delay: int, min_delay: int, adjusted_max_delay: double, enc_origin: string]

In [55]:
df3 = df2.drop(df2.origin)

In [56]:
df3.show()


[Stage 59:==================================================>       (7 + 1) / 8]

+---------+---------+------------------+--------------------+
|max_delay|min_delay|adjusted_max_delay|          enc_origin|
+---------+---------+------------------+--------------------+
|      401|      -15|             601.5|0d403b92007a67a49...|
|      428|      -27|             642.0|ed15ffd6ff72d4444...|
|      414|      -19|             621.0|50725d49d336ca550...|
|      435|      -23|             652.5|53596a1c21dd707b4...|
|      403|      -21|             604.5|ae52e133c0058654f...|
|      353|      -21|             529.5|687e7a185e12ec984...|
|      745|      -27|            1117.5|96f70dbef1131b89b...|
|      135|      -14|             202.5|228f6d2c84eb95ee5...|
|      196|      -15|             294.0|5030551cd5ca2f564...|
|      530|      -30|             795.0|31234537adb52d5df...|
|      236|      -21|             354.0|7a5912423e6c9aea0...|
|      920|      -22|            1380.0|cf4c24e87f3bf7a7b...|
|      808|      -29|            1212.0|dd395d6201581ad21...|
|      2

In [59]:
from pyspark.sql.functions import asc,desc,when, otherwise

ImportError: cannot import name 'otherwise' from 'pyspark.sql.functions' (/home/hdoop/.local/lib/python3.10/site-packages/pyspark/sql/functions.py)

In [65]:
df4 = df.withColumn("Flight_Delays",
                    when(df.delay>360,"Very Long Delays")\
                   .when(df.delay>120,"Long Delays")\
                   .when(df.delay>60,"Short Delays")\
                   .when(df.delay>0,"Tolerabel Delays")\
                   .when(df.delay == 0,"No Delays")\
                   .otherwise("Early")).orderBy(df.origin.desc(),df.delay.desc())

In [66]:
df4.show()

+-------+-----+--------+------+-----------+----------------+
|   date|delay|distance|origin|destination|   Flight_Delays|
+-------+-----+--------+------+-----------+----------------+
|3281505|  475|     139|   YUM|        PHX|Very Long Delays|
|2080600|  347|     206|   YUM|        LAX|     Long Delays|
|2210600|  333|     206|   YUM|        LAX|     Long Delays|
|3291030|  285|     139|   YUM|        PHX|     Long Delays|
|1040600|  267|     206|   YUM|        LAX|     Long Delays|
|1261532|  236|     206|   YUM|        LAX|     Long Delays|
|2230600|  231|     206|   YUM|        LAX|     Long Delays|
|2201605|  230|     139|   YUM|        PHX|     Long Delays|
|2220600|  172|     206|   YUM|        LAX|     Long Delays|
|3180500|  154|     206|   YUM|        LAX|     Long Delays|
|2201530|  135|     206|   YUM|        LAX|     Long Delays|
|1291918|   97|     206|   YUM|        LAX|    Short Delays|
|3021530|   97|     206|   YUM|        LAX|    Short Delays|
|2171605|   95|     139|

In [76]:
ndf= df.select(df.origin,df.delay).where(df.delay>20).groupBy(df.origin).agg(min("delay").alias("min_delay"))

In [94]:
df5 = df.select(df.destination,df.origin,df.delay).join(ndf.select(ndf.min_delay) , on = (ndf.origin == df.origin)&(ndf.min_delay == df.delay))

In [95]:
df5.show()

[Stage 84:===========================================>              (6 + 2) / 8]

+-----------+------+-----+---------+
|destination|origin|delay|min_delay|
+-----------+------+-----+---------+
|        DTW|   ABE|   33|       33|
|        ATL|   ABE|   25|       25|
|        ATL|   ABE|   25|       25|
|        ORD|   ABE|   24|       24|
|        ORD|   ABE|   24|       24|
|        DFW|   ABI|   26|       26|
|        DFW|   ABI|   22|       22|
|        DFW|   ABI|   22|       22|
|        DFW|   ABI|   22|       22|
|        DFW|   ABI|   22|       22|
|        DFW|   ABI|   22|       22|
|        DFW|   ABI|   22|       22|
|        DFW|   ABI|   22|       22|
|        DFW|   ABI|   22|       22|
|        DFW|   ABI|   22|       22|
|        DFW|   ABI|   22|       22|
|        DFW|   ABI|   22|       22|
|        DFW|   ABI|   22|       22|
|        DFW|   ABI|   22|       22|
|        DFW|   ABI|   22|       22|
+-----------+------+-----+---------+
only showing top 20 rows



#### ques 11 onwards

In [96]:

from pyspark.sql.types import *

In [105]:
data = [ (1, 10),
    (1, 12),
    (1, 6),
    (2, 5),
    (2, 6),
    (3, 9),
    (3, 56)]

rdd = spark.sparkContext.parallelize(data)

In [109]:
rdd.groupByKey().mapValues(lambda x: sorted(list(x))).collect()

[(1, [6, 10, 12]), (2, [5, 6]), (3, [9, 56])]

In [110]:
data  = [(1,1),
        (1,2),
        (1,5),
        (2,95),
        (2,3)]
rdd = spark.sparkContext.parallelize(data)

In [112]:
rdd2  = rdd.aggregateByKey(
    (0,0),
    lambda acc,value: (acc[0]+value,acc[1]+1),
    lambda acc1,acc2: (acc1[0]+acc2[0],acc1[1]+acc2[1])
)

In [113]:
rdd2.collect()

[(1, (8, 3)), (2, (98, 2))]

In [120]:
def func(n):
    sum :int = 0
    for i in range(n+1):
        sum +=i
    return sum

In [125]:
rdd3 = rdd2.map(lambda x:(x[0],round(x[1][0]/func(x[1][1]),2)))

In [126]:
rdd3.collect()

[(1, 1.33), (2, 32.67)]